<a href="https://colab.research.google.com/github/Siarzis/ai-tutor/blob/main/numpy_lstm_text_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [2]:
import numpy as np

Create a simple text sequence dataset

In [12]:
np.random.seed(42)

def generate_dataset(num_sequences=100):

    sequences = []

    for _ in range (num_sequences):
        random_number =  np.random.randint(1, 10)
        sample = ['a'] * random_number + ['b'] * random_number +  ['EOS']
        sequences.append(sample)

    return sequences

sequences = generate_dataset()
print(sequences[88])

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'EOS']
